In [14]:
import pandas as pd
from pathlib import Path
from collections import OrderedDict

# Set file paths for input CSVs and output SQL
CSV_DIR = Path('/home/jianwei07/dev/mcomp-projects/it5008_project/doc')  # Assumes CSVs are in the same directory
OUTPUT_SQL_FILE = 'data_draft.sql'

In [15]:
def sql_quote(val):
    """Formats a value for an SQL string, handling NULLs and escaping quotes."""
    if pd.isna(val) or val is None or str(val).strip() == '':
        return 'NULL'
    s = str(val).replace("'", "''")
    return f"'{s}'"

def format_values(row, columns):
    """Formats a list of values from a DataFrame row for an SQL INSERT statement."""
    return ', '.join([sql_quote(row[col]) for col in columns])


In [16]:
print("Reading CSV files...")
menu_df = pd.read_csv(CSV_DIR / 'menu.csv')
registration_df = pd.read_csv(CSV_DIR / 'registration.csv')
staff_df = pd.read_csv(CSV_DIR / 'staff.csv')
orders_df = pd.read_csv(CSV_DIR / 'orders.csv')
print("All CSV files have been read successfully.")

Reading CSV files...
All CSV files have been read successfully.


In [17]:
print("Generating INSERT statements...")

# Dictionary to hold all generated INSERT statements
inserts = OrderedDict()

# --- 4.1. Cuisines --- 
cuisines_from_menu = menu_df['Cuisine'].unique()
cuisines_from_staff = staff_df['Cuisine'].unique()
all_cuisines = pd.Series(pd.concat([pd.Series(cuisines_from_menu), pd.Series(cuisines_from_staff)])).unique()
cuisines_df = pd.DataFrame(all_cuisines, columns=['cuisine_name'])
inserts['cuisines'] = [f"INSERT INTO cuisines (cuisine_name) VALUES ({sql_quote(row['cuisine_name'])});" for _, row in cuisines_df.iterrows()]

# --- 4.2. Menu --- 
menu_df_clean = menu_df[['Item', 'Price', 'Cuisine']].copy()
menu_df_clean = menu_df_clean.rename(columns={'Item': 'item', 'Price': 'price', 'Cuisine': 'cuisine_name'})
inserts['menu'] = [f"INSERT INTO menu (item, price, cuisine_name) VALUES ({format_values(row, ['item', 'price', 'cuisine_name'])});" for _, row in menu_df_clean.iterrows()]

# --- 4.3. Staff --- 
staff_df_clean = staff_df.drop_duplicates(subset=['Staff']).copy()
staff_df_clean = staff_df_clean.rename(columns={'Staff': 'staff_id', 'Name': 'staff_name'})
inserts['staff'] = [f"INSERT INTO staff (staff_id, staff_name) VALUES ({format_values(row, ['staff_id', 'staff_name'])});" for _, row in staff_df_clean.iterrows()]

# --- 4.4. Staff_can_prepare (Junction Table) --- 
staff_cuisine_df = staff_df[['Staff', 'Cuisine']].copy()
staff_cuisine_df = staff_cuisine_df.rename(columns={'Staff': 'staff_id', 'Cuisine': 'cuisine_name'})
inserts['staff_can_prepare'] = [f"INSERT INTO staff_can_prepare (staff_id, cuisine_name) VALUES ({format_values(row, ['staff_id', 'cuisine_name'])});" for _, row in staff_cuisine_df.iterrows()]

# --- 4.5. Registration --- 
reg_from_orders = orders_df[['Phone', 'Firstname', 'Lastname']].rename(columns={'Phone': 'phone', 'Firstname': 'first_name', 'Lastname': 'last_name'}).copy()
reg_from_orders = reg_from_orders[reg_from_orders['phone'].notna() & reg_from_orders['phone'].apply(lambda x: isinstance(x, (str, int)))]
reg_df_clean = pd.concat([registration_df.rename(columns={'Phone': 'phone', 'Firstname': 'first_name', 'Lastname': 'last_name', 'Date': 'reg_date', 'Time': 'reg_time'}), reg_from_orders])
reg_df_clean = reg_df_clean.drop_duplicates(subset=['phone']).copy()
reg_df_clean = reg_df_clean[reg_df_clean['phone'].apply(lambda x: isinstance(x, (str, int)) and len(str(x)) == 8)].copy()
# For records from orders.csv, we don't have reg_date/time, so we'll use a placeholder
reg_df_clean['reg_date'] = reg_df_clean['reg_date'].fillna('1970-01-01')
reg_df_clean['reg_time'] = reg_df_clean['reg_time'].fillna('00:00:00')
inserts['registration'] = [f"INSERT INTO registration (phone, first_name, last_name, reg_date, reg_time) VALUES ({format_values(row, ['phone', 'first_name', 'last_name', 'reg_date', 'reg_time'])});" for _, row in reg_df_clean.iterrows()]

# --- 4.6. Payment_card --- 
payment_card_df = orders_df[orders_df['Payment'] == 'card'][['Card', 'CardType']].drop_duplicates().copy()
payment_card_df = payment_card_df.rename(columns={'Card': 'card_number', 'CardType': 'card_type'})
inserts['payment_card'] = [f"INSERT INTO payment_card (card_number, card_type) VALUES ({format_values(row, ['card_number', 'card_type'])});" for _, row in payment_card_df.iterrows()]

# --- 4.7. Orders --- 
orders_df_header = orders_df.drop_duplicates(subset=['Order']).copy()
orders_df_header = orders_df_header.rename(columns={'Date': 'orders_date', 'Time': 'orders_time', 'Order': 'orders_id', 'Payment': 'payment', 'TotalPrice': 'total_price', 'Phone': 'phone'})
orders_df_header = orders_df_header[['orders_id', 'orders_date', 'orders_time', 'payment', 'total_price', 'phone']].copy()
inserts['orders'] = [f"INSERT INTO orders (orders_id, orders_date, orders_time, payment, total_price, phone) VALUES ({format_values(row, ['orders_id', 'orders_date', 'orders_time', 'payment', 'total_price', 'phone'])});" for _, row in orders_df_header.iterrows()]

# --- 4.8. Order_Items --- 
order_items_df = orders_df.groupby(['Order', 'Item']).agg(
    staff_id=('Staff', 'first'),
    qty=('Item', 'count')
).reset_index()
order_items_df = order_items_df.rename(columns={'Order': 'orders_id', 'Item': 'item', 'Staff': 'staff_id'})
order_items_df = pd.merge(order_items_df, menu_df[['Item', 'Price']].rename(columns={'Item': 'item', 'Price': 'unit_price'}), on='item', how='left')
inserts['order_items'] = [f"INSERT INTO order_items (orders_id, item, staff_id, qty, unit_price) VALUES ({format_values(row, ['orders_id', 'item', 'staff_id', 'qty', 'unit_price'])});" for _, row in order_items_df.iterrows()]

print("All INSERT statements have been generated in memory.")

Generating INSERT statements...
All INSERT statements have been generated in memory.


In [18]:
# --- Section 5: Save to data.sql file ---
with open(OUTPUT_SQL_FILE, 'w') as f:
    f.write("--\n")
    f.write("-- File: data.sql\n")
    f.write("-- Description: INSERT statements for all tables\n")
    f.write("-- Generated automatically by generate_data.py\n")
    f.write("--\n\n")

    for table_name, statement_list in inserts.items():
        if statement_list:
            f.write(f"--\n-- Data for table: {table_name}\n--\n")
            f.write('\n'.join(statement_list))
            f.write('\n\n')
    
    f.write("-- Simple query to show data is successfully inserted\n")
    f.write("SELECT COUNT(*) FROM orders;\n")

print(f"Successfully saved all INSERT statements to {OUTPUT_SQL_FILE}")

Successfully saved all INSERT statements to data_draft.sql
